In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [9]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle['daily_outage_dfs'][6]

(datetime.datetime(2022, 9, 14, 14, 39, 14, 236704, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp   value              fuelType
 0  2022-09-14 00:00:00-06:00  441.00              Intertie
 1  2022-09-14 01:00:00-06:00  472.00              Intertie
 2  2022-09-14 02:00:00-06:00  472.00              Intertie
 3  2022-09-14 03:00:00-06:00  472.00              Intertie
 4  2022-09-14 04:00:00-06:00  472.00              Intertie
 ..                       ...     ...                   ...
 66 2022-09-16 18:00:00-06:00  419.54  3-Day Solar Forecast
 67 2022-09-16 19:00:00-06:00  228.34  3-Day Solar Forecast
 68 2022-09-16 20:00:00-06:00   26.62  3-Day Solar Forecast
 69 2022-09-16 21:00:00-06:00    0.00  3-Day Solar Forecast
 70 2022-09-16 22:00:00-06:00    0.00  3-Day Solar Forecast
 
 [9527 rows x 3 columns])

In [39]:
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
daily

,timeStamp,value,fuelType
0,2022-09-12 00:00:00-06:00,750,Intertie
1,2022-09-13 00:00:00-06:00,0,Intertie
2,2022-09-14 00:00:00-06:00,0,Intertie
3,2022-09-15 00:00:00-06:00,0,Intertie
4,2022-09-16 00:00:00-06:00,0,Intertie
...,...,...,...
86,2022-12-07 00:00:00-07:00,0,Wind
87,2022-12-08 00:00:00-07:00,0,Wind
88,2022-12-09 00:00:00-07:00,0,Wind
89,2022-12-10 00:00:00-07:00,0,Wind


In [10]:
datetime.now(pytz.timezone('America/Edmonton')).date() + relativedelta(days=7)

datetime.date(2022, 9, 21)

In [7]:
 datetime.now(pytz.timezone('America/Edmonton')).date() + relativedelta(months=4, day=1)

datetime.date(2023, 1, 1)

In [9]:
datetime(2022,12,31,0)+relativedelta(days=1,minutes=-5)

datetime.datetime(2022, 12, 31, 23, 55)